# Pandas

Es una biblioteca de Python ampliamente utilizada para la manipulación y análisis de datos. Proporciona estructuras de datos flexibles y poderosas como DataFrame y Series, que permiten almacenar, limpiar, transformar, analizar y manipular grandes volúmenes de datos de manera eficiente. Se utiliza comúnmente en tareas de análisis exploratorio de datos (EDA), procesamiento de datos y preparación de datos para modelos de Machine Learning.

In [ ]:
import numpy as np
import pandas as pd
import pathlib

import time

In [ ]:
PATH_DATA = pathlib.Path(r"../data/")

## Creacion de dataframe

In [ ]:
# Dimensiones del dataframe
N = 2000
M = 10

# Semilla de numpy
np.random.seed(202503)

In [ ]:
# Se genera dataframe con M columnas y N filas numericas
df = pd.DataFrame(
    np.random.normal(0, 1, (N, M)), columns=[f"COL_{i}" for i in range(M)]
)
print(df.shape)
df.head()

In [ ]:
# Se le agregan 5 variables string
for letra in "abcde".upper():
    df[f"COL_{letra}"] = np.random.choice(
        [f"REGION_{i}" for i in range(20)], df.shape[0], replace=True
    )
print(df.shape)
df.head()

In [ ]:
# Se simula un NRO_ID
print(df.shape)
df["NRO_ID"] = np.random.randint(10000000, 30000000, df.shape[0])
df = df.drop_duplicates(subset=["NRO_ID"])
print(df.shape)

# Se genera tipo cliente
df["TYPE"] = np.random.choice(["NUEVO", "BANCO"], df.shape[0], replace=True)
print(df.shape)
df.head()

In [ ]:
# Se genera segundo dataframe solo con fechas
df_dates = pd.DataFrame(
    pd.date_range(start="2024-01-01", periods=420, freq="D"), columns=["DATE_FIELD"]
)
df_dates.tail()

In [ ]:
# Se realiza un cross join
print(df.shape)
print(df.shape[0] * df_dates.shape[0])
df = df.join(df_dates, how="cross")
print(df.shape)
df.head()

In [ ]:
# Se genera una mascara para eliminar filas
filas_a_eliminar = np.random.choice(
    np.arange(df.shape[0]), np.floor(0.2 * df.shape[0]).astype(np.int64), replace=False
)
mask = np.ones(df.shape[0]).astype(np.bool)
mask[filas_a_eliminar] = False

# Se aplica la máscara
print(df.shape)
df = df.iloc[mask, :]
print(df.shape)

In [ ]:
df.describe()

In [ ]:
df.info()

##  Indexación y Slicing

In [ ]:
# Por nombres
df_aux1 = df.loc[
    (
        ((df.COL_1 > 0) | (df.COL_4 > 0))
        & (df.COL_A).isin(["REGION_1", "REGION_7", "REGION_15"])
    ),
    ["DATE_FIELD", "NRO_ID", "TYPE", "COL_3", "COL_4"],
]
print(df_aux1.shape)
df_aux1.head(3)

In [ ]:
# Por indices
df_aux2 = df.iloc[
    np.where(
        ((df.COL_1 > 0) | (df.COL_4 > 0))
        & (df.COL_A).isin(["REGION_1", "REGION_7", "REGION_15"])
    )[0],
    [17, 16, 15, 3, 4],
]
df_aux2.head(3)

In [ ]:
# Seleccionar columnas mediante expresiones regulares
# Ejemplo: columnas que terminan en numero
ls_col_num = df.filter(regex=r".*\d+$").columns
ls_col_num

In [ ]:
# Ejemplo: columnas que terminan en letra
ls_col_str = df.filter(regex=r".*\_[A-Z]$").columns
ls_col_str

In [ ]:
# Ejemplo: columnas ID
ls_cols_id = df.filter(regex=r"(DATE_FIELD|NRO_ID|TYPE)").columns
ls_cols_id

## Operaciones

In [ ]:
# Suma o resta
(df.COL_1 + df.COL_3).head()

In [ ]:
# Suma o resta
df.loc[:, ["COL_1", "COL_3"]].sum(axis=1).head()

In [ ]:
# Sumar todos los elementso de una columna
df.COL_1.sum()

In [ ]:
# Quantiles
df.COL_1.quantile(
    [
        0.1,
        0.5,
        1,
    ]
)

In [ ]:
# 3 grupos por quantiles 0.1, 0.5, 1.
pd.qcut(
    df.COL_1,
    [
        0.0,
        0.1,
        0.5,
        1.0,
    ],
    duplicates="drop",
).head()

In [ ]:
# Desplazamienos
df["COL_1_P"] = df.COL_1.shift(2)
df.head()

## Group by

In [ ]:
# Group by
df.groupby(["NRO_ID"], dropna=False, as_index=False).COL_1.mean().head()

In [ ]:
# Agg
df.groupby(["TYPE", "COL_A"], dropna=False, as_index=False).agg(
    AVG_COL_1=("COL_1", "mean"),
    MAX_COL_7=("COL_7", "max"),
).head()

In [ ]:
# Transform
df["COL_1_SUM_BY_TYPE_AND_COL_A"] = df.groupby(
    ["TYPE", "COL_A"], dropna=False, as_index=False
).COL_1.transform("sum")
df.head()

In [ ]:
# rolling (window)
df["COL_1_ROLL"] = df.groupby(["NRO_ID"]).COL_1.transform(
    lambda x: x.rolling(window=3).sum()
)
df.head()

## Pivotar

In [ ]:
# Pivotar
df_aux = df.pivot_table(
    values="COL_3", index=["COL_A"], columns="TYPE", aggfunc="mean", dropna=False
).head()
df_aux = df_aux.reset_index()
df_aux

In [ ]:
# Wide to long
df_aux = df_aux.melt(
    id_vars="COL_A", value_vars=["BANCO", "NUEVO"], var_name="TYPE", value_name="VALOR"
)
df_aux

## Concatenar dataframes

In [ ]:
df_aux = df.iloc[df.shape[0] // 2 :, :]
print(df_aux.shape)
df_aux_2 = pd.concat([df_aux, df], axis=0, ignore_index=True)
print(df_aux_2.shape)
df_aux_2.head()

## Ordenar DataFrame

In [ ]:
df = df.sort_values(by=["NRO_ID", "DATE_FIELD"], ascending=[True, True])
df.head()

## Joins

In [ ]:
dict_reg_ciu = {"REGION": [], "CIUDAD": []}
for i in range(20):
    dict_reg_ciu["REGION"].append(f"REGION_{i}")
    dict_reg_ciu["CIUDAD"].append(f"CIUDAD_{i}")
df_reg_ciu = pd.DataFrame(dict_reg_ciu)
df_reg_ciu.head()

In [ ]:
print(df.shape)
df_3 = df.merge(df_reg_ciu, how="inner", left_on="COL_A", right_on="REGION")
print(df_3.shape)
df_3.head()

## Ejemplo calculo percentiles

In [ ]:
# Previo
def percentiles_1(df, feature, k):
    limites = pd.Series([i / k for i in range(k + 1)])
    return df[feature].rank(pct=1).apply(lambda x: (limites >= x).argmax())


# Posterior
def percentiles_2(df, feature, k):
    limites = np.linspace(0, 1, k + 1)
    return (df[feature].rank(pct=1).values.reshape((-1, 1)) <= limites).argmax(axis=1)

In [ ]:
df_prueba = pd.DataFrame(np.random.normal(0, 1, (600000, 4)))
start_time = time.time()
percentiles_1(df_prueba, 2, 10)
print(time.time() - start_time)

In [ ]:
start_time = time.time()
percentiles_2(df_prueba, 2, 10)
print(time.time() - start_time)

## Escritura y lectura

In [ ]:
# Escritura a parquet (necesita package adicional)
df.to_parquet(PATH_DATA.joinpath(r"prueba.parquet"), index=False)
df.head(2)

In [ ]:
# Lectura parquet (necesita package adicional)
df = pd.read_parquet(PATH_DATA.joinpath(r"prueba.parquet"))
df.head(2)

In [ ]:
# Escritura a csv
df.to_csv(PATH_DATA.joinpath(r"prueba.csv"), sep=";", index=False, float_format="%.8f")
df.head()

In [ ]:
# Lectura csv
df = pd.read_csv(PATH_DATA.joinpath(r"prueba.csv"), sep=";")
df.head(2)

In [ ]:
# Escritura a csv (comprimido)
df.to_csv(
    PATH_DATA.joinpath(r"prueba.csv.gz"), sep=";", index=False, float_format="%.8f"
)

In [ ]:
# Lectura csv (comprimido)
df = pd.read_csv(PATH_DATA.joinpath(r"prueba.csv.gz"), sep=";")
df.head(2)

In [ ]:
# Escritura a xlsx (necesita package adicional)
df.to_excel(PATH_DATA.joinpath(r"prueba.xlsx"), index=False)

In [ ]:
# Lectura xlsx (necesita package adicional)
df = pd.read_excel(PATH_DATA.joinpath(r"prueba.xlsx"))